In [8]:
import numpy as np
from sklearn.decomposition import FastICA, PCA
from shogun import SOBI,RealFeatures

import pylab as pl
from amuse import amuse
from scipy.io import wavfile
from sklearn import preprocessing    
import matplotlib.pyplot as plt
from jade import jadeR
import pywt
#read sources
fs1, s1 = wavfile.read('si30.wav')
fs2, s2 = wavfile.read('Small-talk30.wav')
fs3, s3 = wavfile.read('taboo+topic.wav')
#5 sec of sources

s1 = s1[0:220500]
s2 = s2[0:220500]
s3 = s3[0:220500]
s1=s1[:,0]
s2=s2[:,0]
s3=s3[:,0]
S0 = np.c_[s1, s2, s3]

#S0 = S0[~(S0==0).all(1)]
d=S0.std(axis=0)
S =S0/ d  

#s= preprocessing.normalize(s, norm='l2')
A = np.array([[3.2,4.5,6.7],[1.7,2.4,3],[2,3.6,1.8]])  #Matrix mixture. Merges the sources

x = np.dot(S, A.T) #observed signal

Wavelet=pywt.Wavelet('db3')
level_max=0
coeffs1 = pywt.wavedec(x[:,0], Wavelet, level=level_max)
coeffs2 = pywt.wavedec(x[:,1], Wavelet, level=level_max)
coeffs3 = pywt.wavedec(x[:,2], Wavelet, level=level_max)
x_=np.c_[coeffs1[0], coeffs2[0],coeffs3[0]]

amuse1 = amuse(x_.T, 3, 1)
s_hat = (amuse1.sources).T #estimate sources. 1/8 size of the original signal
W = amuse1.W #separation matrix

ica = FastICA(n_components=3)
S_ica = ica.fit_transform(x)  # 重构信号
# PCA模型
pca = PCA(n_components=3)
S_pca = pca.fit_transform(x)  # 基于PCA的成分正交重构信号源

#jade
jade=np.asarray(jadeR(x.T,verbose=False))
J=np.dot(jade,x.T)

#sobi
sep = SOBI()
mixed_signals = RealFeatures((S.T).astype(np.float64))             
signals = sep.apply(mixed_signals)

So = signals.get_feature_matrix()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(S)

plt.show()
plt.plot(x)
plt.show()

In [10]:
from sklearn.metrics import mean_squared_error
import copy
def get_error(ground_truth,signalmat):
    # we want to try every possible signal order
    g=ground_truth.T
    s=signalmat.T
    l = len(s) # number of signals
    
    best_err =mean_squared_error(g,s)
    for j in range(l):        
        temp = copy.deepcopy(s[j])
        for i in range(j,l-1):
            s[i]=copy.deepcopy(s[i+1])
            s[i+1]=copy.deepcopy(temp)

            err = mean_squared_error(g,s)
            if err > best_err:
                best_err = err
    return best_err

In [11]:
import math

print(get_error(S,S_ica))
print(get_error(S,S_pca))
print(get_error(S,s_hat)) #amuse
print(get_error(S,J.T))   #jade
print(get_error(S,So.T))  #sobi

1.0014272619301527
37.04988159875823
3114.673171969118
2.0497478865054815
2.0066945650885866


In [ ]:
0.999878461056
132.826856575
13.3647274612
2.66372264982

1.0014125579
73.067925018
11.1346606895
2.68414535111

0.999968211396
35.1467833345
3797.13541275
2.04895349219